In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
import netCDF4 as nc 
import warnings
import xarray as xr
import pickle
import concurrent.futures
# Suppress specific warnings using filterwarnings()
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [ ]:
# Import the 'caffeine' library for preventing screen from going to sleep
import caffeine
# Enable caffeine mode with the option to keep the display on
caffeine.on(display=True)  

In [ ]:
# Read data from an HDF5 file into a DataFrame
data = pd.read_hdf('Save/Track.h5',key='/table') 
# Open a NetCDF file as an xarray Dataset for segmentation mask
mask = xr.open_dataset('Save/Mask_Segmentation_TB.nc')
# Open a NetCDF file as an xarray Dataset for TB and TAU data
ds = xr.open_dataset('Save/TAU_TB.nc')

In [ ]:
# Get rid of cells that have only one feature in them
data = data.groupby('cell').filter(lambda x : (x['cell'].count()>1)).reset_index(drop=True)

# Get pixel level infromation on brightness temperature, optical thickness and longitude/latitude

In [ ]:
# Convert xarray DataArrays to NumPy arrays for 'tb', 'tau', and 'mask'
tb = np.array(ds.Tb.values)
tau = np.array(ds.Tau.values)
mask = np.array(mask.segmentation_mask.values)

In [ ]:
# Extract 'frame' and 'feature' columns as NumPy arrays from the DataFrame
frames = data['frame'].to_numpy() 
features = data['feature'].to_numpy()
# Print the shapes of the 'features' and 'frames' arrays
print(features.shape, frames.shape)

In [ ]:
# Function to retrieve brightness temperature data for every feature
def get_tb(a, b):
    # The get_tb function retrieves 'tb' data based on certain conditions specified by 'a' and 'b':
    # a - time frame, b - unique feature id
    try:
        # Retrieve 'tb' data where 'mask' matches the specified condition
        res = tb[a][np.where(mask[a] == b)].tolist()
        return res  # Return the resulting 'tb' data
    except:
        pass  # Handle exceptions by returning None

In [ ]:
# Function to retrieve optical thickness data for every feature
def get_tau(a,b):
    try:
        res = tau[a][np.where(mask[a]==b)].tolist()
        return res
    except:
        pass

In [ ]:
# Function to retrieve latitude data for every feature
def get_lats(a,b):
    try: 
        indices = np.where(mask[a] == b)
        lats = ds['lat'][indices[0]].values.tolist()
        return lats
    except:
        pass

In [ ]:
# Function to retrieve longitude data for every feature
def get_lons(a,b):
    try:
        indices = np.where(mask[a] == b)
        lons = ds['lon'][indices[1]].values.tolist()
        return lons
    except:
        pass

In [ ]:
dict_tb = {}
def process_feature(args):
    a, b = args
    return b, get_tb(a, b)
# Process 'frame' and 'feature' pairs in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_feature = {executor.submit(process_feature, (a, b)): b for a, b in zip(frames, features)}
    for future in concurrent.futures.as_completed(future_to_feature):
        feature = future_to_feature[future]
        try:
            res = future.result()
        except Exception as exc:
            print(f'Feature {feature} generated an exception: {exc}')
        else:
            dict_tb[res[0]] = res[1]

In [ ]:
# Define the path to the pickle file
pickle_file_path = 'Save/pickles/tb_dict.pkl'
# Save the dictionary to the pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(dict_tb, pickle_file)

In [ ]:
del dict_tb

In [ ]:
dict_tau = {}
def process_feature(args):
    a, b = args
    return b, get_tau(a, b)
# Process 'frame' and 'feature' pairs in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_feature = {executor.submit(process_feature, (a, b)): b for a, b in zip(frames, features)}
    for future in concurrent.futures.as_completed(future_to_feature):
        feature = future_to_feature[future]
        try:
            res = future.result()
        except Exception as exc:
            print(f'Feature {feature} generated an exception: {exc}')
        else:
            dict_tau[res[0]] = res[1]

In [ ]:
# Define the path to the pickle file
pickle_file_path = 'Save/pickles/tau_dict.pkl'
# Save the dictionary to the pickle file 
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(dict_tau, pickle_file)

In [ ]:
del dict_tau

In [ ]:
dict_lons = {}
def process_feature(args):
    a, b = args
    return b, get_lons(a, b)
# Process 'frame' and 'feature' pairs in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_feature = {executor.submit(process_feature, (a, b)): b for a, b in zip(frames, features)}
    for future in concurrent.futures.as_completed(future_to_feature):
        feature = future_to_feature[future]
        try:
            res = future.result()
        except Exception as exc:
            print(f'Feature {feature} generated an exception: {exc}')
        else:
            dict_lons[res[0]] = res[1]

In [ ]:
# Define the path to the pickle file
pickle_file_path = 'Save/pickles/lons_dict.pkl'
# Save the dictionary to the pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(dict_lons, pickle_file)

In [ ]:
del dict_lons

In [ ]:
dict_lats = {}
def process_feature(args):
    a, b = args
    return b, get_lats(a, b)
# Process 'frame' and 'feature' pairs in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_feature = {executor.submit(process_feature, (a, b)): b for a, b in zip(frames, features)}
    for future in concurrent.futures.as_completed(future_to_feature):
        feature = future_to_feature[future]
        try:
            res = future.result()
        except Exception as exc:
            print(f'Feature {feature} generated an exception: {exc}')
        else:
            dict_lats[res[0]] = res[1]

In [ ]:
# Define the path to the pickle file
pickle_file_path = 'Save/pickles/lats_dict.pkl'
# Save the dictionary to the pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(dict_lats, pickle_file)

In [ ]:
del dict_lats

In [ ]:
data